In [21]:
import os
import pprint
import asyncio
import aiohttp
from dotenv import load_dotenv
from googlesearch import search
from fake_useragent import UserAgent
import requests
from bs4 import BeautifulSoup

### 電視資訊爬取

In [6]:
def fetch_page(url):
    # Placeholder for fetching a page; you'll replace this with actual fetching logic
    return requests.get(url)

def process_url(url, parser):
    try:
        response = fetch_page(url)
        if response.status_code == 200:
            result = parser(response.text, url)
            return result
        else:
            print(f"Failed to fetch {url}: Status code {response.status_code}")

    except Exception as e:
        print(f"Error fetching {url}: {e}")
        

def parse_page_for_tv(html_content, url):
    try:
        soup = BeautifulSoup(html_content, 'html.parser')
        text_content_list = soup.find_all('p')
        # print(text_content_list)
        soup = BeautifulSoup(str(text_content_list), 'html.parser')
        text = soup.getText()
        text = clean_text(text)
        return text
    
    except Exception as e:
        print(f"Error parsing {url}: {e}")


def clean_text(text):
    text = text.split('\n')

    # remove unwanted string
    text = [x.strip() for x in text]
    text = [x for x in text if x != '']
    text = [x for x in text if x != ' ']
    text = [x for x in text if x != "\n"]
    text = [x for x in text if x != ", "]
    text = [x for x in text if x != ","]
    text = [x for x in text if x != "'"]

    merged_text = '\n'.join(text)
    merged_text = merged_text.replace('\t', '')
    merged_text = merged_text.replace(', ', '')

    return merged_text


def write_to_file(text, file_name):
    if text is None:
        return
    
    with open(file_name, 'w') as f:
        f.write(text)

In [3]:
def run():
    url = input("Enter the URL: ")
    site = url.split('/')[2]
    print(url)
    output = process_url(url, parser=parse_page_for_tv)
    # write_to_file(output, f"./result/{site}.txt")
    print(output)

In [36]:
run()

https://agirls.aotter.net/post/62519
[
2023/08/13
by 電獺少女編輯室
這次小編來跟大家講解智慧電視怎麼挑，老規矩不會有品牌懶人包，而是跟大家介紹選購時該注意的事項，但歡迎各家廠商置入自薦，小編絕對會好好介紹 >.^雖然好像講了句廢話，但電視相比於其他電器產品，各廠牌對於不同等級的電視價位級距切得相當精準，可不是那種捏一下錢包，忍一個月不喝飲料就能接受的差距，所以抓好預算再來挑選電視比較好除了價錢外，接著要抉擇的是電視尺寸，並不是說大就好，電視大個幾吋不只價錢可能翻倍，家中又不一定擺得下，而且真的要大，也大不過現在越來越多人挑選的微型投影機（想看投影機推薦也可以敲碗小編）至於該挑多大，可以參考由 Sony 提出的建議參考網站：電視的建議觀賞距離是多遠？ | Sony TW現在買電視不一定只有追劇看電影，也有不少電玩玩家喜歡將 PS5、NS 等家用主機接上大螢幕，享受極致的娛樂體驗或與家人同樂，所以相對於畫質，可能更需要考慮高幀數或有補幀技術的電視；另外如果專門拿來看電影的用戶，除了畫質還會注重到喇叭的表現，所以先想想你的需求，再挑選適合你的電視才是最省時省力的這些，就是貴到市面上看不到、貴到買不起、超貴、很貴、平民選擇，小資族眼裡是這樣區分的結束（被打）真的要探討這幾種面板技術，多到可以再寫三篇文章也不為過，容小編用普通人也看得懂的方式介紹Micro LED：技術發展的目標，還沒量產，所以很貴，也看不太到mini LED：顯色鮮豔、亮度高、壽命長、省電、光暈會發散，技術還在發展所以很貴OLED：單點發色、能做到真正的純黑、亮度不高、有烙印問題壽命較短、顯色漂亮、技術逐漸成熟，目前畫質派的主流QLED：傳統 LED 的變體、亮度比 OLED 高、光暗對比優秀、顯色漂亮、壽命較 OLED 長、技術成熟，是小資族也能享受的奢華選擇UHD：根本不是面板而是畫質的稱呼，但很多廠牌會專門列出一個 4K UHD 系列來嚇唬大家，讓你以為有 D 的意志就很厲害一樣，沒有，就只是沒跟你說是傳統主流的液晶面板而已，技術相當成熟，俗又大碗現在稱智慧電視而不是螢幕，就是因為多了智慧聯網功能介面，讓你可以像使用手機一樣的操作電視，以往要追劇可能還要額外配備電視盒才能觀看 Netflix、YouTube 等串流平台，但現在主流的智慧電視都有內建

### 關鍵字相關網頁爬取

In [51]:
def gather_by_keyword(keyword, num_results=5):
    result = search(keyword, num_results)
    result = list(result)

    for url in result:
        site = url.split('/')[2]
        print(url)
        output = process_url(url, parser=parse_page_for_tv)
        write_to_file(output, f"./result/{site}.txt")

# sum all txt in result folder
def concact_txt():
    text = ""
    for file in os.listdir("./result"):
        with open(f"./result/{file}", 'r') as f:
            text += f.read()
    return text


def save_txt(text, file_name):
    with open("./result/" + file_name + ".txt", 'w') as f:
        f.write(text)


# save all txt in result folder to one txt
def concact_and_save():
    text = concact_txt()
    save_txt(text, "sum_result")

In [56]:
gather_by_keyword("如何挑選電視？", num_results=7)
# concact_and_save()

In [40]:
# read sum_result.txt
with open("./result/sum_result.txt", 'r') as f:
    text = f.read()

list = text.split('。')

In [41]:
for sentence in list:
    print(sentence)
    print("-----")

[需要幫助？關注我們請輸入您的會員 Email 及密碼新顧客？ 創建會員帳戶忘記密碼？ 重置密碼請輸入您的 Email：還記得密碼？ 返回直接登入請在下方填寫有關信息：已經是會員？ 由此登入踏入智能科技的年代，大部份的傳統家電亦重新定位，例如過往屬於家庭電器的核心，提供一家大細娛樂的電視機，亦升級為智能產品，不再單純接收本地電視台的節目，而是可以利用內置的作業系統連接互聯網，直接串流播放更多影音平台的內容，帶來更多新的體驗
-----
選購電視機的時候，大家會發現有很多專有名詞及規格，例如 4K UHD／FULL HD、HDR、曲面／平面、作業系統、尺寸等等，令人難以理解當中的意思，所以今次特別為大家進行出簡單解說，讓你輕鬆掌握選購要點
-----
自 2018 年開始，大部份電視機品牌都相繼推出具有 4K UHD 畫質的電視機型號，而當中的 4K 就是指畫面解析度達到 3840×2160 或 4096×2160 像素達到 829 萬的規格
-----
當 4K 與 FULL HD 比較的話，4K 畫面精細度會高達 4 倍，而伴隨 4K 出現的 UHD，就是指 HDR 高動態範圍，現時該技術的主流標準是 HDR 10
-----
當應用於畫面的時候，就可以把影像的亮度提高到 1000 nits
-----
簡單說明，就是令影像呈現得更有層次，色彩更為飽和及鮮豔當然這些技術都會直接反映在價格之上，因此一般來說 4K UHD 的電視會比 FULL HD 電視貴，但隨着此項技術普及，4K UHD 的電視價格，近來也逐漸由近萬元一部，降價至數千不等
-----
除了畫質技術之外，部份品牌的機型採用了全新推出的曲面螢幕設計，雖然有絕佳的視覺體驗，但因為製作成本提高，售價也隨之而偏高，另外也因為螢幕是曲面的關係，只有坐在正中的人才可以享受最佳效果，稍為偏坐一旁的觀眾就會受到角度的影響，相反，平面螢幕的電視機就沒有以上的問題，雖然呈現出來的畫質沒有曲面的好，但電視機主要是為一家人提供娛樂，除非你是獨自一人生活，否則未必個個都可以享受曲面電視帶來的極致的畫面，所以大家在選購的時候，還是要留意平面及曲面的分別
-----
除了以上的規格及技術之外，大家都會以螢幕的尺寸為選購考量，一般而言，大部份人都會在客廳安裝大尺寸的電視機，如房間使用的話，會選擇 43 吋這類較小型的電視，但原來未

### 商品資訊爬取

In [54]:
def fetch_page_advanced(url):
    ua = UserAgent()
    user_agent = ua.random
    headers = {'User-Agent': user_agent}

    # add headers to request
    html_content = requests.get(url, headers=headers)
    return html_content

def parse_page_for_momo(html_content, url):
    try:
        soup = BeautifulSoup(html_content, 'html.parser')
        tab_list = soup.find_all(class_ = 'TabContentD selected')
        # print(tab_list)

        product_list = []
        for tab in tab_list:
            product = parse_momo_for_product(str(tab), url)
            product_list  = product_list + product

        return product_list
    
    except Exception as e:
        print(f"Error parsing {url}: {e}")
        
def parse_momo_for_product(html_content, url):
    try:
        soup = BeautifulSoup(html_content, 'html.parser')
        # Find all <li> tags
        products = soup.find_all('li')

        product_list = []
        # Iterate through each product
        for product in products:
            product_name = product.find('h3', class_='prdName').get_text(strip=True)  # Extract product name
            product_price = product.find('b', class_='money').get_text(strip=True)  # Extract product price
            product_link = 'https://www.momoshop.com.tw' + product.find('a')['href'] # Extract product link
            product = {
                'name': product_name,
                'price': product_price,
                'link': product_link
            }
            product_list.append(product)
        return product_list
    
    except Exception as e:
        print(f"Error parsing {url}: {e}")

def print_html_context(html_content):
    soup = BeautifulSoup(html_content, 'html.parser')
    print(soup.prettify())

In [55]:
# print out the html content
url = input("Enter the URL: ")  
html_content = fetch_page_advanced(url).text
product_list = parse_page_for_momo(html_content, url)


In [56]:
product_list

[{'name': '【Philips 飛利浦】55吋4K android聯網液晶顯示器(55PUH8257)',
  'price': '12999',
  'link': 'https://www.momoshop.com.tw/goods/GoodsDetail.jsp?i_code=11932211&str_category_code=2900900120&mdiv=2900900120-bt_9_002_01&ctype=B&Area=DgrpCategory'},
 {'name': '【RANSO 聯碩】43型FHD低藍光液晶顯示器(RF-43VA3)',
  'price': '9990',
  'link': 'https://www.momoshop.com.tw/goods/GoodsDetail.jsp?i_code=8822321&str_category_code=2900900120&mdiv=2900900120-bt_9_002_01&ctype=B&Area=DgrpCategory'},
 {'name': '【JVC】50型 Apple認證AirPlay2 4K HDR 飛輪體感連網液晶顯示器(50TG)',
  'price': '9999',
  'link': 'https://www.momoshop.com.tw/goods/GoodsDetail.jsp?i_code=10628869&str_category_code=2900900120&mdiv=2900900120-bt_9_002_01&ctype=B&Area=DgrpCategory'},
 {'name': '【小米】55型智慧顯示器 A2 (L55M7-EATWN)',
  'price': '10999',
  'link': 'https://www.momoshop.com.tw/goods/GoodsDetail.jsp?i_code=11616098&str_category_code=2900900120&mdiv=2900900120-bt_9_002_01&ctype=B&Area=DgrpCategory'},
 {'name': '【HERAN 禾聯】50型4K HDR 智慧聯網液晶顯示器(HD-50SDF43)',
  'p